# 参数管理

在选择了架构并设置了超参数后，我们就进入了训练阶段。
此时，我们的目标是找到使损失函数最小化的模型参数值。
经过训练后，我们将需要使用这些参数来做出未来的预测。
此外，有时我们希望提取参数，以便在其他环境中复用它们，
将模型保存下来，以便它可以在其他软件中执行，
或者为了获得科学的理解而进行检查。

之前的介绍中，我们只依靠深度学习框架来完成训练的工作，
而忽略了操作参数的具体细节。
本节，我们将介绍以下内容：

* 访问参数，用于调试、诊断和可视化。
* 参数初始化。
* 在不同模型组件间共享参数。

(**我们首先看一下具有单隐藏层的多层感知机。**)


In [46]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))        #定义网络，包含一个线性层4*8，后接ReLU函数，再接8*1
X = torch.rand(size=(2, 4))                                             #定义2*4的随机张量
net(X)                                                                  #传入网络

tensor([[-0.1590],
        [-0.1423]], grad_fn=<AddmmBackward0>)

## [**参数访问**]

我们从已有模型中访问参数。
当通过`Sequential`类定义模型时，
我们可以通过索引来访问模型的任意层。
这就像模型是一个列表一样，每层的参数都在其属性中。
如下所示，我们可以检查第二个全连接层的参数。


In [47]:
print(net[2].state_dict())                                              #查看第二个全连接层的参数，包含权重和偏置

OrderedDict([('weight', tensor([[ 0.2423,  0.2586,  0.3070, -0.0649,  0.2978, -0.0879, -0.0884,  0.2572]])), ('bias', tensor([-0.2781]))])


输出的结果告诉我们一些重要的事情：
首先，这个全连接层包含两个参数，分别是该层的权重和偏置。
两者都存储为单精度浮点数（float32）。
注意，参数名称允许唯一标识每个参数，即使在包含数百个层的网络中也是如此。

### [**目标参数**]

注意，每个参数都表示为参数类的一个实例。
要对参数执行任何操作，首先我们需要访问底层的数值。
有几种方法可以做到这一点。有些比较简单，而另一些则比较通用。
下面的代码从第二个全连接层（即第三个神经网络层）提取偏置，
提取后返回的是一个参数类实例，并进一步访问该参数的值。


In [48]:
print(type(net[2].bias))            #输出第二个全连接层偏置的类型
print(net[2].bias)                  #输出第二个全连接层的偏置
print(net[2].bias.data)             #输出第二个全连接层的偏置的数值

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.2781], requires_grad=True)
tensor([-0.2781])


参数是复合的对象，包含值、梯度和额外信息。
这就是我们需要显式参数值的原因。
除了值之外，我们还可以访问每个参数的梯度。
在上面这个网络中，由于我们还没有调用反向传播，所以参数的梯度处于初始状态。


In [49]:
net[2].weight.grad == None          #判断参数是否处于初始状态

True

### [**一次性访问所有参数**]

当我们需要对所有参数执行操作时，逐个访问它们可能会很麻烦。
当我们处理更复杂的块（例如，嵌套块）时，情况可能会变得特别复杂，
因为我们需要递归整个树来提取每个子块的参数。
下面，我们将通过演示来比较访问第一个全连接层的参数和访问所有层。


In [50]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])  #输出第一层参数的名字和参数类型
print(*[(name, param.shape) for name, param in net.named_parameters()])     #输出第一第二层的名字和擦书类型

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


这为我们提供了另一种访问网络参数的方式，如下所示。


In [51]:
net.state_dict()['2.bias'].data                 #输出网络参数，字典对应标签

tensor([-0.2781])

### [**从嵌套块收集参数**]

让我们看看，如果我们将多个块相互嵌套，参数命名约定是如何工作的。
我们首先定义一个生成块的函数（可以说是“块工厂”），然后将这些块组合到更大的块中。


In [52]:
def block1():                                           #定义第一个块
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),    #定义一个4*8的线性层，后跟ReLU函数
                         nn.Linear(8, 4), nn.ReLU())    #定义一个8*4的线性层，后更ReLU函数

def block2():                                           #定义第二个块
    net = nn.Sequential()                               #定义网络
    for i in range(4):                                  #循环四次
        # 在这里嵌套                                    
        net.add_module(f'block {i}', block1())          #对这个网络循环嵌套4个block1
    return net                                          #返回该block2 网络

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))        #定义rgnet网络，在block2网络后，跟一个4*1的线性层
rgnet(X)                                                #调用该rgnet网络

tensor([[-0.1557],
        [-0.1557]], grad_fn=<AddmmBackward0>)

[**设计了网络后，我们看看它是如何工作的。**]


In [53]:
print(rgnet)                                            #输出网络的架构

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


因为层是分层嵌套的，所以我们也可以像通过嵌套列表索引一样访问它们。
下面，我们访问第一个主要的块中、第二个子块的第一层的偏置项。


In [54]:
rgnet[0][1][0].bias.data                        #输出第一个主块的第二个子块的第一层的8个偏置

tensor([ 0.4849,  0.3711,  0.4911, -0.2650, -0.4473, -0.1811, -0.3456, -0.3136])

## 参数初始化

知道了如何访问参数后，现在我们看看如何正确地初始化参数。
我们在 :numref:`sec_numerical_stability`中讨论了良好初始化的必要性。
深度学习框架提供默认随机初始化，
也允许我们创建自定义初始化方法，
满足我们通过其他规则实现初始化权重。


默认情况下，PyTorch会根据一个范围均匀地初始化权重和偏置矩阵，
这个范围是根据输入和输出维度计算出的。
PyTorch的`nn.init`模块提供了多种预置初始化方法。


### [**内置初始化**]

让我们首先调用内置的初始化器。
下面的代码将所有权重参数初始化为标准差为0.01的高斯随机变量，
且将偏置参数设置为0。


In [55]:
def init_normal(m):                                 #初始化，标准差为0.01的高斯随机变量 偏置设为0
    if type(m) == nn.Linear:                        #如果是线性层
        nn.init.normal_(m.weight, mean=0, std=0.01) #初始化权重，
        nn.init.zeros_(m.bias)                      #初始化权重为0
net.apply(init_normal)                              #调用初始化
net[0].weight.data[0], net[0].bias.data[0]          #输出第一层线性层的权重和偏置

(tensor([ 0.0018,  0.0069, -0.0116, -0.0071]), tensor(0.))

我们还可以将所有参数初始化为给定的常数，比如初始化为1。


In [56]:
def init_constant(m):                               #初始化
    if type(m) == nn.Linear:                        #判断是否为线性层
        nn.init.constant_(m.weight, 1)              #权重统一设置为1
        nn.init.zeros_(m.bias)                      #偏置设置为0
net.apply(init_constant)                            #调用该初始化
net[0].weight.data[0], net[0].bias.data[0]          #输出第一层线性层的权重和偏置

(tensor([1., 1., 1., 1.]), tensor(0.))

我们还可以[**对某些块应用不同的初始化方法**]。
例如，下面我们使用Xavier初始化方法初始化第一个神经网络层，
然后将第三个神经网络层初始化为常量值42。


In [57]:
def xavier(m):                                      #Xavier初始化方法
    if type(m) == nn.Linear:                        #如果是线性层
        nn.init.xavier_uniform_(m.weight)           #各层的激活值和状态梯度的方差在传播过程中的方差保持一致
def init_42(m):                                     #常数初始化
    if type(m) == nn.Linear:                        #如果判断是线性层
        nn.init.constant_(m.weight, 42)             #初始化权重为常数42

net[0].apply(xavier)                                #网络1层使用xavier方法
net[2].apply(init_42)                               #网络3层使用常数初始化方法
print(net[0].weight.data[0])                        #输出模型权权重
print(net[2].weight.data)                           #输出第3层全连接侧层权重初始化参数

tensor([ 0.0462, -0.1853, -0.4060,  0.4990])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


### [**自定义初始化**]

有时，深度学习框架没有提供我们需要的初始化方法。
在下面的例子中，我们使用以下的分布为任意权重参数$w$定义初始化方法：

$$
\begin{aligned}
    w \sim \begin{cases}
        U(5, 10) & \text{ 可能性 } \frac{1}{4} \\
            0    & \text{ 可能性 } \frac{1}{2} \\
        U(-10, -5) & \text{ 可能性 } \frac{1}{4}
    \end{cases}
\end{aligned}
$$


同样，我们实现了一个`my_init`函数来应用到`net`。


In [58]:
def my_init(m):                                                         #自己来初始化一种权重
    if type(m) == nn.Linear:                                            #判断是不是线性层
        print("Init", *[(name, param.shape)                             #输出权重的形状
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)                             #设置权重初始化区间
        m.weight.data *= m.weight.data.abs() >= 5                       #判断绝对值是不是>5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-0.0000,  5.2994,  0.0000, -6.5366],
        [-0.0000, -6.6709,  0.0000, -0.0000]], grad_fn=<SliceBackward0>)

注意，我们始终可以直接设置参数。


In [66]:
net[0].weight.data[:] += 1          #对所有权重+1
net[0].weight.data[0, 0] = 42       #对指定参数设为42
net[0].weight.data[0]               #输出参数内容

tensor([42.0000, 44.0000,  6.2107,  6.4305])

## [**参数绑定**]

有时我们希望在多个层间共享参数：
我们可以定义一个稠密层，然后使用它的参数来设置另一个层的参数。


In [69]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)                            #定义一个8*8的共享层（稠密层）
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),     #定义模型，线性层4*7跟激活函数，跟共享层，跟RelU函数，跟共享层，跟ReLU激活函数，跟线性层8*1
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)                                              #模型传入参数
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])   #判断参数是否相同
net[2].weight.data[0, 0] = 100                          #对一个层改变，这个共享层会同时改变
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


这个例子表明第三个和第五个神经网络层的参数是绑定的。
它们不仅值相等，而且由相同的张量表示。
因此，如果我们改变其中一个参数，另一个参数也会改变。
你可能会思考：当参数绑定时，梯度会发生什么情况？
答案是由于模型参数包含梯度，因此在反向传播期间第二个隐藏层
（即第三个神经网络层）和第三个隐藏层（即第五个神经网络层）的梯度会加在一起。


## 小结

* 我们有几种方法可以访问、初始化和绑定模型参数。
* 我们可以使用自定义初始化方法。

## 练习

1. 使用 :numref:`sec_model_construction` 中定义的`FancyMLP`模型，访问各个层的参数。
1. 查看初始化模块文档以了解不同的初始化方法。
1. 构建包含共享参数层的多层感知机并对其进行训练。在训练过程中，观察模型各层的参数和梯度。
1. 为什么共享参数是个好主意？


[Discussions](https://discuss.d2l.ai/t/1829)
